<a href="https://colab.research.google.com/github/pranay8297/Stable-Diffusion-Experiments/blob/main/Backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

{}

In [2]:
import numpy as np
import torch

In [5]:
import torchvision
from torchvision import datasets

In [6]:
ds = torchvision.datasets.MNIST(root = './', train = True, transform = None, download = True)

100%|██████████| 9912422/9912422 [00:00<00:00, 96425333.96it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 46234997.64it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 24590622.98it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4339528.19it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [7]:
x, y = ds.data, ds.targets

In [8]:
x_train, y_train, x_valid, y_valid = x[:50000], y[:50000], x[50000:], y[50000:]

In [9]:
x_train.shape, x_valid.shape

(torch.Size([50000, 28, 28]), torch.Size([10000, 28, 28]))

In [10]:
x_train, x_valid = x_train.flatten(1, -1), x_valid.flatten(1, -1)

In [12]:
class Mse():
  def __call__(self, inp, target):
    self.inp = inp
    self.target = target
    self.out = torch.mean((target - inp)**2)
    return self.out

  def backward(self):
    '''
      (2/N)*torch.sum()
    '''
    n = self.inp.shape[0]
    dj_da = (self.inp - self.target)*(2/n)
    self.inp.g = dj_da

def Relu():
  def __call__(self, inp):
    self.inp = inp
    self.out = inp.clamp_min(0.)
    return self.out

  def backward(self):

    '''da/dz - '''
    da_dz = (self.inp > 0.0).float()
    '''dl/dz = dl/da * da/dz'''
    dl_da = self.out.g
    dl_dz = da_dz * dl_da
    self.inp.g = dl_dz

def Linear():
  def __init__(self, w, b):
    self.w = w
    self.b = b

  def __call__(self, x):
    self.inp = x
    self.out = self.w.T @ x + self.b
    return self.out

  def backward(self):
    '''dz/dw - [dz/dw1 dz/dw2, dz/dw3]
       dl/dz - self.out.g
    '''
    dl_dz = self.out.g # (bs * 1)
    dz_dw = self.inp.T # n_features * bs
    dl_dw = dz_dw @ dl_dz# n_parameters * 1 [dl/da1 * da1/dz1* dz1/dw1 + dl/da2 * da2/dz2 * dz2/dw1]
    self.w.g = dl_dw

    dl_db = dl_dz.sum(0)
    self.b.g = dl_db